getting dataset from previous runs and prep for training:

(since natural image flopped horendously, try if synthetic data is more efficient; i guess it really is)

In [11]:
# list datasets in org

curr_org = "distill-lab"

from huggingface_hub import HfApi

api = HfApi()
datasets = api.list_datasets(
    author=curr_org,
)

res = [i for i in datasets]

In [23]:
ids = [i.id for i in res]

artist_ids = [i for i in ids if "ai-distill-artist_v1-" in i and "eagle.library" in i]
artist_ids

['distill-lab/nai-distill-artist_v1-batch0_eagle.library',
 'distill-lab/nai-distill-artist_v1-batch1_eagle.library']

In [41]:
import unibox as ub

# df = ub.loads("hf://distill-lab/nai-distill-artist_v1-batch0_eagle.library")
df = ub.loads("hf://distill-lab/nai-distill-artist_v1-batch1_eagle.library")

2025-04-14 17:31:14 [INFO] loads: Loading from hf://distill-lab/nai-distill-artist_v1-batch1_eagle.library


README.md:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Setting num_proc from 8 to 5 for the train split as it only contains 5 shards.


Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [42]:
# filter star to be 2; df is huggingface dataset

df = df.filter(lambda x: x["star"] == 2)
len(df)

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

240

In [43]:
import os

save_dir = "../data/artist_v1-batch01_kepts"
os.makedirs(save_dir, exist_ok=True)

for item in df:
    img_bytes = item["image"]["bytes"]
    filename = item["filename"]

    if img_bytes is None:
        continue  # skip missing

    # Save raw bytes as .png
    with open(os.path.join(save_dir, filename), "wb") as f:
        f.write(img_bytes)

write tags:

this is a **COPY** of the original functionality in trainlib

In [49]:
from PIL import Image
import numpy as np
import gzip
import json

from io import BytesIO

def byteize(alpha):
    alpha = alpha.T.reshape((-1,))
    alpha = alpha[:(alpha.shape[0] // 8) * 8]
    alpha = np.bitwise_and(alpha, 1)
    alpha = alpha.reshape((-1, 8))
    alpha = np.packbits(alpha, axis=1)
    return alpha



class LSBExtractor:
    def __init__(self, data):
        self.data = byteize(data[..., -1])
        self.pos = 0

    def get_next_n_bytes(self, n):
        n_bytes = self.data[self.pos:self.pos + n]
        self.pos += n
        return bytearray(n_bytes)

    def read_32bit_integer(self):
        bytes_list = self.get_next_n_bytes(4)
        return int.from_bytes(bytes_list, 'big') if len(bytes_list) == 4 else None


def extract_image_metadata(image: Image.Image) -> dict:
    # Convert PIL Image to numpy array
    image_array = np.array(image.convert("RGBA"))
    
    # Verify image format
    if image_array.shape[-1] != 4 or len(image_array.shape) != 3:
        raise ValueError("Image must be in RGBA format")

    # Extract data
    reader = LSBExtractor(image_array)
    magic = "stealth_pngcomp"
    read_magic = reader.get_next_n_bytes(len(magic)).decode("utf-8")
    
    if magic != read_magic:
        raise ValueError("Invalid magic number")

    # Get and decompress JSON data
    json_len = reader.read_32bit_integer() // 8
    compressed_json = reader.get_next_n_bytes(json_len)
    json_data = json.loads(gzip.decompress(compressed_json).decode("utf-8"))

    # Handle nested Comment field if present
    if "Comment" in json_data and isinstance(json_data["Comment"], str):
        json_data["Comment"] = json.loads(json_data["Comment"])

    return json_data

In [57]:
img_paths = ub.ls(save_dir, ub.IMG_FILES)

for curr_path in img_paths:
    try:
        img = Image.open(curr_path)
        metadata = extract_image_metadata(img)
        prompt = metadata.get("Description", "")
        caption = prompt.replace("{{7010 umou_(umouawa)}} {amagasa_yun} [[[ningen_mame]]] {{{very aesthetic}}} classic_(zildjian33) year_2024 aran_(aran0127), ", "")
        caption = caption.replace(", no text, best quality, very aesthetic, absurdres", "").replace(", general", "")
        ub.saves([caption], curr_path.replace(".png", ".txt"), debug_print=False)
    except Exception as e:
        print(f"Error processing {curr_path}: {e}")

2025-04-14 17:48:34 [INFO] ls: Listing contents of ../data/artist_v1-batch01_kepts


Listing local files: 0files [00:00, ?files/s]

In [58]:
len(img_paths)

490

upload to huggingface just in case:

In [59]:

import os
import glob
import json
from datasets import Dataset, Features, Value, Image as HFImage

def create_image_dataset(base_dir, mode="json"):
    """
    Creates a HuggingFace dataset from images and optional label/metadata files.

    Args:
        base_dir (str): Directory containing image files and corresponding label files
        mode (str): One of ['json', 'txt', 'image_only']

    Returns:
        datasets.Dataset: A HuggingFace dataset
    """
    assert mode in {"json", "txt", "image_only"}, f"Unsupported mode: {mode}"

    images = []
    filenames = []
    extra_columns = {}

    # Collect image paths
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.webp']
    image_files = []
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(base_dir, ext)))
        image_files.extend(glob.glob(os.path.join(base_dir, ext.upper())))

    for image_file in image_files:
        base_name = os.path.splitext(os.path.basename(image_file))[0]

        images.append(image_file)
        filenames.append(os.path.basename(image_file))

        if mode == "txt":
            txt_file = os.path.join(base_dir, f"{base_name}.txt")
            label = ""
            if os.path.exists(txt_file):
                try:
                    with open(txt_file, 'r') as f:
                        label = f.read().strip()
                except Exception as e:
                    print(f"Error reading {txt_file}: {e}")
            extra_columns.setdefault("label", []).append(label)

        elif mode == "json":
            json_file = os.path.join(base_dir, f"{base_name}.json")
            if os.path.exists(json_file):
                try:
                    with open(json_file, 'r') as f:
                        data = json.load(f)
                    for k, v in data.items():
                        extra_columns.setdefault(k, []).append(str(v))
                    for k in extra_columns:
                        if k not in data:
                            extra_columns[k].append("")
                except Exception as e:
                    print(f"Error reading {json_file}: {e}")
                    for k in extra_columns:
                        extra_columns[k].append("")
            else:
                for k in extra_columns:
                    extra_columns[k].append("")

    # Assemble dataset dict
    dataset_dict = {
        "image": images,
        "filename": filenames,
        **extra_columns
    }

    # Define features
    features = {
        "image": HFImage(),
        "filename": Value("string")
    }
    for k in extra_columns:
        features[k] = Value("string")

    dataset = Dataset.from_dict(dataset_dict)
    dataset = dataset.cast(Features(features))
    return dataset

if __name__ == "__main__":
    base_dir = "/local/yada/apps/SimpleTuner-a/data/artist_v1-batch01_kepts"
    mode = "txt"  # options: "json", "txt", "image_only"

    dataset = create_image_dataset(base_dir, mode=mode)
    
    print(f"Dataset created with {len(dataset)} images")
    print(f"Columns: {dataset.column_names}")

    dataset.push_to_hub("incantor/qft-nai4-artist-b1b2", private=True)


Casting the dataset:   0%|          | 0/490 [00:00<?, ? examples/s]

Dataset created with 490 images
Columns: ['image', 'filename', 'label']


Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/245 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Map:   0%|          | 0/245 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]